In [1]:
import matplotlib
#matplotlib.use('Agg')

In [2]:
import os
import datetime
import time
import copy
import shutil
import sys
sys.path.append('/cnrm/pastel/USERS/lecestres/analyse/')
from function_read import *
from HWs_detection import *
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, shiftgrid

import calendar
import locale
locale.setlocale( locale.LC_ALL , 'en_US' )
from netCDF4 import num2date, date2num 
import netCDF4
from math import sin, cos, sqrt, atan2, radians
from sklearn.neighbors import DistanceMetric
from math import radians

from joblib import Parallel, delayed
import joblib

In [3]:
### EXPERIENCE NAME
expname = "ocean_reanalysis_GREP"
#expname = "sst_retroprevision_sys7"

### PERCENTILE THRESHOLD
percent_thresh = 95
#percent_thresh = 90

### MINIMAL DURATION OF A HW
duration_min = 5
#duration_min = 3

### REGION OF EXPERIENCE
reg_name = "global"
#reg_name = 'lat30_70-lon-40_0'
#reg_name = 'lat-20_30-lon-30_10'
if reg_name == 'global':
    lats_bnds = [-90,90]
    lons_bnds = [-180,180]    
elif reg_name == 'lat30_70-lon-40_0':
    lats_bnds = [30,70]
    lons_bnds = [-40, 0]
elif reg_name == 'lat-20_30-lon-30_10':
    lats_bnds = [-20, 20]
    lons_bnds = [-30, 10]
nlat = lats_bnds[1]-lats_bnds[0]
nlon = lons_bnds[1]-lons_bnds[0]

### YEARS
if expname == 'ocean_reanalysis_GREP':
    end_year=2018
    start_year=1993
elif expname == 'sst_retroprevision_sys7':
    end_year=2016
    start_year=1993
nyear=end_year-start_year+1

### SEASON
season = "NDJFMAM"
if season == 'NDJFMAM':
    nday = 211
    season_start_day = [11,1] #1stNov
    season_end_day = [5,31] #31stMay
elif season == 'DJF':
    nday = 90
ndayseas = nday//duration_min +1

if expname == "ocean_reanalysis_GREP":    
    ### NUMBER OF MEMBS
    nmemb = 1
    chosen_memb = [0] #should never change
    memb_str = 'memb' + str(chosen_memb[0])
    
    ### CROSS VALIDATION
    cv = False
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'
        
elif expname == "sst_retroprevision_sys7":    
    ### NUMBER OF MEMBS
    nmemb = 1 #For now keep just one memb by one
    chosen_memb = [21] #to be filled in numerical order
    if len(chosen_memb) != nmemb:
        raise(ValueError)
    if nmemb>1:
        memb_str = 'memb' + str(chosen_memb[0]) + '-' + str(chosen_memb[-1])
    else:
        memb_str = 'memb' + str(chosen_memb[0])
    
    ### CROSS VALIDATION
    cv = True
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'

### PARAMETERS
parameters_str = reg_name + "_" + season + "_" + cv_str + '_percent%i'%(percent_thresh) + '_daymin%i'%(duration_min) +  "ref%i-%i"%(start_year, end_year) + '_' + memb_str

### NAME OF THE VARIABLES IN THE NC FILES
#varname = 'HWMI_' + parameters_str
varname = 'subHW_' + parameters_str

In [4]:
pathHWMI = '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/' + expname + '/' + memb_str + '/' + varname + '/'
files = glob(pathHWMI + '*.nc')
files.sort()

In [5]:
files

['/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0_1993_1994.nc',
 '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0_1994_1995.nc',
 '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0_1995_1996.nc',
 '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0_1996_1997.nc',
 '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/memb0/HWMI_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0/HWMI_global_NDJFMAM_no

In [6]:
obsregyearslst = []
for file in files:
    varf=netCDF4.Dataset(file)
    varf.variables[varname]
    vararray, lats_reg, lons_reg = extract_array(varf, varname, ndayseas, np.array(lons_bnds), np.array(lats_bnds),  start_time = 0, level=0)
    obsreg = vararray[:, np.newaxis, :, :]
    maskobs = obsreg.mask
    obsregyearslst.append(obsreg)

obsregyears = np.ma.array(obsregyearslst)

varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)
varf.variables[varname].shape :  (1, 1, 180, 360)


In [7]:
start_time = time.time()
args = (expname, reg_name, memb_str, parameters_str, start_year, lats_reg, lons_reg)
HWampliobsmembyear, HWstartobsmembyear, HWendobsmembyear, fieldobslstmembyear = calc_HW_MY(obsregyears, maskobs, lats_reg, lons_reg, args, allowdist=1)
print('Total time for detection : ', time.time() - start_time)

0
(array([0, 0, 0, ..., 0, 0, 0]), array([ 13,  13,  13, ..., 175, 175, 175]), array([  0,   1,   2, ..., 218, 219, 220]))
/cnrm/pastel/USERS/lecestres/NO_SAVE/data/ocean_reanalysis_GREP/memb0/Ampli_Fields_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0/Ampli_Fields_global_NDJFMAM_notCV_percent95_daymin5ref1993-2018_memb0_1993_1994.nc
ndayseas :  1
(434, 1)
(434, 1)
time for year  1993  :  3.8596889972686768
(array([0, 0, 0, ..., 0, 0, 0]), array([ 13,  13,  13, ..., 174, 174, 174]), array([  0,   1,   2, ..., 230, 231, 232]))


KeyboardInterrupt: 